In [1]:
#GOOGLE SCHOLAR SEARCH OF KEY WORDS
from bs4 import BeautifulSoup
import pandas as pd
import requests

url= 'https://scholar.google.com/scholar'

#set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' 
    #english
}

params = {
    #english
    'hl': 'en',

    'start': 0,
    'q': 'circular economy waste household' # maybe add 'trash' 'garbage' 'individual'
}

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm

# extract doi's from url
def extract_doi_from_webpage(url):
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            doi_elements = soup.find_all(string=re.compile(r'\b(10\.\d{4,}(?:\.\d+)*/\S+)\b'))
            
            if doi_elements:
                dois = []
                for element in doi_elements:
                    match = re.findall(r'\b(10\.\d{4,}(?:\.\d+)*/\S+)\b', str(element))
                    if match:
                        #added 'https://doi.org/' in front so that iit can be callable
                        dois.extend([f"https://doi.org/{doi}" for doi in match])
                
                return dois 
            else:
                return None
        else:
            return None
    except requests.exceptions.RequestException as e:
        return None

# loop through pages for n pages
n = 10

results = pd.DataFrame(columns=['titles', 'links', 'authors', 'dois'])

for i in tqdm(range(n)):
    params['start'] = i * 10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    # get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    titles = [title[:title.find('[') - 1] if title.find('[') != -1 else title for title in titles]
    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]

    # get doi, if it is None, keep link ^ as url of article 
    dois = [extract_doi_from_webpage(link) for link in links]
    dois_links = [doi[0] if doi else None for doi in dois]
    final_links = [doi if doi else link for doi, link in zip(dois_links, links)]

    
    results = pd.concat([results, pd.DataFrame({'titles': titles, 'links': final_links, 'authors': authors, 'dois': dois_links})])

results.to_csv('article_info3.csv', index=False)


  0%|          | 0/10 [00:00<?, ?it/s]

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [3]:
print(results.head)

<bound method NDFrame.head of                                                titles  \
0   [HTML][HTML] Consumers in a circular economy: ...   
1   [HTML][HTML] Circular economy and household e-...   
2   [HTML][HTML] Household organic waste: Integrat...   
3   [HTML][HTML] Waste prevention, energy recovery...   
4   [HTML][HTML] Potential for circular economy in...   
..                                                ...   
5   Towards a circular economy: A case study of wa...   
6   Assessing the Outcomes of Circular Economy and...   
7   [PDF][PDF] Effective solid waste management in...   
8   [HTML][HTML] Co-production in solid waste mana...   
9   [HTML][HTML] Waste treatment company decision-...   

                                                links  \
0   https://www.sciencedirect.com/science/article/...   
1   https://www.sciencedirect.com/science/article/...   
2   https://www.sciencedirect.com/science/article/...   
3   https://www.sciencedirect.com/science/article/...   


In [3]:
# basic scraping function

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests

def pdf_download(url_to_search, file_name):
    driver = webdriver.Chrome() #initialize web-finder
    main_page_url = 'https://sci-hub.ru/' #open sci-hub
    driver.get(main_page_url)
    time.sleep(1)
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)
    
    try:
        # Find download link
        download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
        onclick_attribute = download_button.get_attribute('onclick')
    except:
        driver.quit()
        return url_to_search

    
    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    download_link = 'https://sci-hub.ru' + pdf_url
    # Download the PDF using requests
    response = requests.get(download_link)
    with open(f'pdfs/{file_name}.pdf', 'wb') as file:
       file.write(response.content)
    
    driver.quit()

    return None

In [6]:
#LOOP THROUGH URLS to downalod by calling the pdf_download function
import os
data = pd.read_csv('article_info3.csv')
list_error_links = []  # Define an empty list to store error links if encountered

# Function to check if a file exists at a given path
def file_exists(file_path):
    return os.path.exists(file_path)

# Loop through each row in the DataFrame 'data'
for index, row in data.iterrows():
    pdf_file_path = f'pdfs/{index}.pdf'  # Define the path for the PDF file
    if file_exists(pdf_file_path):
        print("PDF already downloaded")
    else:
        if row['links'] in list_error_links:
            print('Error already encountered')
        else:
            url_returned = pdf_download(row['links'], '{index}.pdf')  
            if url_returned is not None:
                list_error_links.append(url_returned)
                print(f'Error encountered for {url_returned}')

PDF already downloaded


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="request"]"}
  (Session info: chrome=120.0.6099.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7B3AA4D02+56194]
	(No symbol) [0x00007FF7B3A104B2]
	(No symbol) [0x00007FF7B38B76AA]
	(No symbol) [0x00007FF7B39016D0]
	(No symbol) [0x00007FF7B39017EC]
	(No symbol) [0x00007FF7B3944D77]
	(No symbol) [0x00007FF7B3925EBF]
	(No symbol) [0x00007FF7B3942786]
	(No symbol) [0x00007FF7B3925C23]
	(No symbol) [0x00007FF7B38F4A45]
	(No symbol) [0x00007FF7B38F5AD4]
	GetHandleVerifier [0x00007FF7B3E1D5BB+3695675]
	GetHandleVerifier [0x00007FF7B3E76197+4059159]
	GetHandleVerifier [0x00007FF7B3E6DF63+4025827]
	GetHandleVerifier [0x00007FF7B3B3F029+687785]
	(No symbol) [0x00007FF7B3A1B508]
	(No symbol) [0x00007FF7B3A17564]
	(No symbol) [0x00007FF7B3A1